In [ ]:
from google.colab import auth,output

from google.auth import default
creds, _ = default()
auth.authenticate_user()
print('Authenticated')
project_id = 'c4-group-datagov-dev'
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

Authenticated


In [ ]:
import pandas as pd
pd.options.display.max_colwidth = None
pd.options.display.max_rows = 6

In [ ]:
full_df = client.query('''
WITH MULTI_BEM AS (SELECT
    distinct country_key,SUB_CLASS_KEY,
    BEM_CLASS_key,
  FROM `c4-gdw-prd.products_referential.d_bem_product_barcode`
)

SELECT
  distinct
  COUNTRY_KEY,
  ITEM_DESC,
  BARCODE,
  B.BEM_CLASS_KEY,
  D.BEM_CLASS_DESC_FR
FROM
  `c4-gdw-prd.products_referential.d_bem_product_barcode` B
LEFT JOIN
  `c4-gdw-prd.products_referential.d_bem_product_sub_class` AS C
ON
  B.BEM_SUB_CLASS_KEY=C.BEM_SUB_CLASS_KEY
LEFT JOIN
  `c4-gdw-prd.products_referential.d_bem_product_class` AS D
ON
  B.BEM_CLASS_KEY=D.BEM_CLASS_KEY
LEFT JOIN
  `c4-gdw-prd.products_referential.d_bem_product_grp_class` AS E
ON
  B.BEM_GRP_CLASS_KEY=E.BEM_GRP_CLASS_KEY
LEFT JOIN
  `c4-gdw-prd.products_referential.d_bem_product_department` AS F
ON
  B.BEM_DEPARTMENT_KEY=F.BEM_DEPARTMENT_KEY
LEFT JOIN
  `c4-gdw-prd.products_referential.d_bem_product_sector` AS G
ON
  B.BEM_SECTOR_KEY=G.BEM_SECTOR_KEY
WHERE
  COUNTRY_KEY IN ("FRA", "BEL")
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%GESTION%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%GAZ%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%REMB%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%CARBURANT%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%CAISSE%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%LIVRAISON%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%EMBAL%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%COMMANDE%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%ECONAMAT%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%ACTUALISATION%"
  AND D.BEM_CLASS_DESC_FR NOT LIKE "%RAYON%"
  AND REGEXP_CONTAINS(barcode, r'^[0-9]+$')
  AND NOT REGEXP_CONTAINS(barcode, r'^(R|CHA)')
  AND B.SUB_CLASS_KEY IN (
    SELECT SUB_CLASS_KEY
    FROM MULTI_BEM
    GROUP BY 1
    HAVING COUNT(*) > 1
    )
    AND B.bem_class_key IN (
    SELECT bem_class_key
    FROM `c4-gdw-prd.products_referential.d_bem_product_barcode`
    WHERE bem_sector_key = '1'
    GROUP BY bem_class_key
    ORDER BY COUNT(*) DESC
    LIMIT 4
)
GROUP BY
  1,
  2,
  3,
  4,
  5
ORDER BY
  2,
  4
            ''').to_dataframe()

In [ ]:
full_df

,COUNTRY_KEY,ITEM_DESC,BARCODE,BEM_CLASS_KEY,BEM_CLASS_DESC_FR
0,FRA,6X75CL BDX ROSE CH.ORANGER,3354900912381,1050,VINS
1,FRA,075 CTE GASC.ROSE SIESTE JEAN,3273591001412,1050,VINS
2,FRA,"1,5L POUILLY VINZELLE",3490871704003,1050,VINS
...,...,...,...,...,...
368887,BEL,ŒUFS DE PÂQUES/PAASEITJES,6953038,1424,CONFISERIE SAISONNIÈRE
368888,BEL,ŒUFS DE PÂQUES/PAASEITJES,5430003476178,1424,CONFISERIE SAISONNIÈRE
368889,BEL,ŒUFS DE PÂQUES/PAASEITJES,16953039,1424,CONFISERIE SAISONNIÈRE


In [ ]:
import re
import string

def data_cleaning(strings):

    strings = strings.lower().strip()

    strings = strings.replace('\'',' ')
    strings = strings.replace('/',' ')

    strings = re.sub(r'[^\w\s]', ' ', strings)

    # Remplacer les caractères spéciaux par un espace
    text_normalized = re.sub('[^A-Za-z ,éêèîôœàâ]+', ' ', strings)

    return text_normalized

full_df['ITEM_DESC'] = full_df['ITEM_DESC'].apply(data_cleaning)
full_df['BEM_CLASS_DESC_FR'] = full_df['BEM_CLASS_DESC_FR'].apply(data_cleaning)

full_df

,COUNTRY_KEY,ITEM_DESC,BARCODE,BEM_CLASS_KEY,BEM_CLASS_DESC_FR
0,FRA,x cl bdx rose ch oranger,3354900912381,1050,vins
1,FRA,cte gasc rose sieste jean,3273591001412,1050,vins
2,FRA,l pouilly vinzelle,3490871704003,1050,vins
...,...,...,...,...,...
368887,BEL,œufs de pâques paaseitjes,6953038,1424,confiserie saisonnière
368888,BEL,œufs de pâques paaseitjes,5430003476178,1424,confiserie saisonnière
368889,BEL,œufs de pâques paaseitjes,16953039,1424,confiserie saisonnière


In [ ]:
cats = full_df.BEM_CLASS_DESC_FR.unique().tolist()
cats

['vins', 'confiserie de sucre', 'maquillage', 'confiserie saisonnière']

In [ ]:
dico = {"rg": "rouge","rges" : "rouge","rge": "rouge", "rse": "rose" ,"rs" : "rose", "ch": "chardonnay","bl": "blanc", "bdx": "Bordeaux", "vdt": "vin de table", 'vdp': "vin de pays","blc": "blanc", "bib": "bag in box"}

def standardization(x):
  liste = x.split(' ')
  for i in range(len(liste)) :
    if liste[i] in dico.keys():
      liste[i] = dico[liste[i]]
  return ' '.join(liste)


full_df.ITEM_DESC[(full_df.BEM_CLASS_KEY == "1050")]  = full_df.ITEM_DESC[(full_df.BEM_CLASS_KEY == "1050")].apply(standardization)

full_df

,COUNTRY_KEY,ITEM_DESC,BARCODE,BEM_CLASS_KEY,BEM_CLASS_DESC_FR
0,FRA,x cl Bordeaux rose chardonnay oranger,3354900912381,1050,vins
1,FRA,cte gasc rose sieste jean,3273591001412,1050,vins
2,FRA,l pouilly vinzelle,3490871704003,1050,vins
...,...,...,...,...,...
368887,BEL,œufs de pâques paaseitjes,6953038,1424,confiserie saisonnière
368888,BEL,œufs de pâques paaseitjes,5430003476178,1424,confiserie saisonnière
368889,BEL,œufs de pâques paaseitjes,16953039,1424,confiserie saisonnière


In [ ]:
import nltk
nltk.download('stopwords')

import plotly.express as px
import sys
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set(rc={'figure.figsize':(14.7,10.27)})
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
from nltk.corpus import stopwords

def remove_stop_words(strings, stops):
    list_item_desc = strings.split(' ')
    cleaned_list = [word for word in list_item_desc if word not in stops]
    return ' '.join(cleaned_list)

en_stops = set(stopwords.words('english'))
fr_stops = set(stopwords.words('french'))
measurments = set(['oz','kg','g','lb','mg','l','cl','ml','tsp','tbsp',"cm","x", 'cte'])
fr_stops.update(measurments)

#full_df['ITEM_DESC'] = full_df['ITEM_DESC'].apply(lambda x: remove_stop_words(x, en_stops))
full_df['ITEM_DESC'] = full_df['ITEM_DESC'].apply(lambda x: remove_stop_words(x, fr_stops))

full_df

,COUNTRY_KEY,ITEM_DESC,BARCODE,BEM_CLASS_KEY,BEM_CLASS_DESC_FR
0,FRA,Bordeaux rose chardonnay oranger,3354900912381,1050,vins
1,FRA,gasc rose sieste jean,3273591001412,1050,vins
2,FRA,pouilly vinzelle,3490871704003,1050,vins
...,...,...,...,...,...
368887,BEL,œufs pâques paaseitjes,6953038,1424,confiserie saisonnière
368888,BEL,œufs pâques paaseitjes,5430003476178,1424,confiserie saisonnière
368889,BEL,œufs pâques paaseitjes,16953039,1424,confiserie saisonnière


In [ ]:
#!python -m spacy download fr_core_news_lg

In [ ]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin
import spacy

# Load spaCy pretrained model that we downloaded before
nlp = spacy.load("fr_core_news_lg")

def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    docs = DocBin()

    for doc, label in nlp.pipe(data, as_tuples=True):
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(target_file)
    return docs

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(full_df["ITEM_DESC"].values, full_df["BEM_CLASS_DESC_FR"].values, test_size=0.3)

make_docs(list(zip(X_train, y_train)), "/content/drive/Shareddrives/RefBEM_Project/spacy/train.spacy", cats = cats)
make_docs(list(zip(X_valid, y_valid)), "/content/drive/Shareddrives/RefBEM_Project/spacy/valid.spacy", cats = cats)

In [ ]:
import spacy
spacy.require_gpu()

True

In [ ]:
#!python -m spacy debug config config.cfg

In [ ]:
from spacy.cli.train import train as spacy_train

config_path = "/content/drive/Shareddrives/RefBEM_Project/spacy/config.cfg"
output_model_path = "/content/drive/Shareddrives/RefBEM_Project/spacy/output/spacy_textcat"
spacy_train(
    config_path,
    output_path=output_model_path,
    overrides={
        "paths.train": "/content/drive/Shareddrives/RefBEM_Project/spacy/train.spacy",
        "paths.dev": "/content/drive/Shareddrives/RefBEM_Project/spacy/valid.spacy",
    },
)

ℹ Saving to output directory:
/content/drive/Shareddrives/RefBEM_Project/spacy/output/spacy_textcat
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.19        0.00    0.00
  0     200         25.57       35.60    0.36
  0     400         15.55       60.29    0.60
  0     600         11.14       71.48    0.71
  0     800          8.84       77.88    0.78
  0    1000          7.47       82.23    0.82
  0    1200          6.62       85.48    0.85
  0    1400          5.58       87.92    0.88
  0    1600          4.85       89.34    0.89
  0    1800          4.29       90.61    0.91
  0    2000          4.04       91.77    0.92
  0    2

In [ ]:
def Pipeline(strings : str):
  strings = data_cleaning(strings)
  strings = standardization(strings)

  fr_stops = set(stopwords.words('french'))
  measurments = set(['oz','kg','g','lb','mg','l','cl','ml','tsp','tbsp',"cm","x", 'cte'])
  fr_stops.update(measurments)
  strings =  remove_stop_words(strings, fr_stops)

  return strings

In [ ]:
import spacy

trained_nlp = spacy.load("/content/drive/Shareddrives/RefBEM_Project/spacy/output/spacy_textcat/model-best")


text = "oeuf rose"

text = Pipeline(text)
doc = trained_nlp(text)

doc.cats

NameError: ignored